# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
import numpy as np

In [14]:
df = pd.read_csv("homepage_actions.csv")
df.head()


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [10]:
# Is ID a unique identifier?
pd.Series(df['id']).is_unique

False

In [11]:
# What actions can be taken?
df['action'].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [39]:
# Create dummies for click and view
dummies = pd.get_dummies(df['action'])
df['click'] = dummies['click']
df['view'] = dummies['view']
df.head(10)


,timestamp,id,group,action,click,view
0,2016-09-24 17:42:27.839496,804196,experiment,view,0,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,0,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,0,1
3,2016-09-24 19:59:02.646620,671993,control,view,0,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,0,1
5,2016-09-24 20:32:25.712659,681598,experiment,view,0,1
6,2016-09-24 20:39:03.248853,522116,experiment,view,0,1
7,2016-09-24 20:57:20.336757,349125,experiment,view,0,1
8,2016-09-24 20:58:01.948663,349125,experiment,click,1,0
9,2016-09-24 21:00:12.278374,560027,control,view,0,1


In [23]:
# Split into experimental and control datasets

experiment = df.loc[df['group']=='experiment']
control = df.loc[df['group']=='control']

,timestamp,id,group,action,click,view
3,2016-09-24 19:59:02.646620,671993,control,view,0,1
9,2016-09-24 21:00:12.278374,560027,control,view,0,1
23,2016-09-25 00:25:14.141290,281985,control,view,0,1
24,2016-09-25 01:14:48.603202,407864,control,view,0,1
25,2016-09-25 02:16:11.046654,342984,control,view,0,1


In [25]:
experiment = experiment.groupby(['id']).sum()
control = control.groupby(['id']).sum()

,click,view
id,,
182994,1,1
183089,0,1
183248,1,1
183515,0,1
183524,0,1


In [30]:
# Did anyone 'click' without 'view'? This would not be possible.  If so, remove them

control.loc[(control['click']==1) & (control['view']==0)]

# This does not seem to be the case

,click,view
id,,


In [35]:
# Look at the average clicks and views for each group

print('Comparison of Experiment and Control Group')
print(f"Mean Views: \tcontrol {round(np.mean(control['view']),2)} \texperiment {round(np.mean(experiment['view']),2)}")
print(f"Mean Clicks: \tcontrol {round(np.mean(control['click']),2)} \texperiment {round(np.mean(experiment['click']),2)}")

Comparison of Experiment and Control Group
Mean Views: 	control 1.0 	experiment 1.0
Mean Clicks: 	control 0.28 	experiment 0.31


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [40]:
#Your code here
import flatiron_stats as fs

# H0: There is no difference in the click rate between the experimental and control groups
# H1: There is a higher click rate on the experimental group

alpha = 0.05

p = fs.p_value_welch_ttest(control['click'], experiment['click'], two_sided=False)
p

0.004466402814337078

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [41]:
#Your code here
experiment_clicks = np.mean(control['click'])*len(experiment)
experiment_clicks
        


838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [43]:
#Your code here

std = np.sqrt(len(experiment)*np.mean(control['click'])*(1-np.mean(control['click'])))

z = (np.mean(experiment['click'])*len(experiment) - np.mean(control['click'])*len(experiment))/std
z


3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [46]:
#Your code here
from scipy.stats import norm

p = 1-norm.cdf(z)
p


0.00012486528006949715

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.